In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [37]:
df = pd.read_csv('./crawling_data/감성대화말뭉치(최종데이터)_Training.csv', encoding='cp949') #미리 토큰화한 감성 문서 활용
df = df.drop_duplicates().iloc[:-1]

C:\Users\eunah.lee\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
len(df)

40879

In [39]:
sa = df[['사람문장1', '감정_대분류']]
sa.columns = ['Text', 'Label']

In [41]:
tmp = sa[:10000]

In [42]:
# 문자 빼고 지움(정규표현식)
# 문장 자르기
import kss
import re

UNKNOWN_WORD = "...하"
cnt = 0

def OnlyWords(s):
    result = re.sub('[^ A-Za-z가-힣]', '', s)
    return result

def sentence_split(x):
    global cnt
    if UNKNOWN_WORD not in x:
        cnt += 1
        if cnt % 10 ==0:
            print(cnt)
        return kss.split_sentences(OnlyWords(x))
    else:
        cnt += 1
        if cnt % 10 ==0:
            print(cnt)
        return kss.split_sentences(OnlyWords(x).replace(UNKNOWN_WORD, ".."))
    
tmp['text_tokenized'] = tmp['Text'].apply(sentence_split)
# sa['text_tokenized']=sa['Text'].apply(\
#         lambda x: kss.split_sentences(OnlyWords(x)) if UNKNOWN_WORD not in x else kss.split_sentences(OnlyWords(x).replace(UNKNOWN_WORD,"..")))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
222

C:\Users\eunah.lee\AppData\Local\Temp/ipykernel_11376/4261903678.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['text_tokenized'] = tmp['Text'].apply(sentence_split)


In [43]:
from konlpy.tag import Okt  
okt=Okt() 
# 명사, 형용사, 동사
def n_a_v_selection(sent_list):
    sent_tokens=[]
    for sentence in sent_list:
        pos_result=okt.pos(sentence, stem=True)
        filtered_result=[pair[0] for pair in pos_result if (pair[1]=='Alpha')or(pair[1]=='Noun')or(pair[1]=='Verb')or(pair[1]=='Adjective') or(pair[1]=='Adverb')]
        sent_tokens.append(filtered_result)
    return sent_tokens

# 명사, 형용사
def n_a_selection(sent_list):
    sent_tokens=[]
    for sentence in sent_list:
        pos_result=okt.pos(sentence, stem=True)
        filtered_result=[pair[0] for pair in pos_result if (pair[1]=='Alpha')or(pair[1]=='Noun')or(pair[1]=='Adjective') or(pair[1]=='Adverb')]
        sent_tokens.append(filtered_result)
    return sent_tokens
        
# 명사만
def noun_selection(sent_list):
    sent_tokens=[]
    for sentence in sent_list:
        sent_tokens.append(okt.nouns(sentence))
    return(sent_tokens)


tmp2['text_tokenized']=tmp2['text_tokenized'].apply(lambda x: n_a_v_selection(x))

C:\Users\eunah.lee\AppData\Local\Temp/ipykernel_11376/2469903079.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['text_tokenized']=tmp['text_tokenized'].apply(lambda x: n_a_v_selection(x))


In [62]:
# 불용어 처리
import json

with open('korean_stopword.json',encoding='utf8') as f:
    stopwords = json.load(f)

stopwords.extend(['안','수','거','왜','것','후','에서','고','이다','는','한','안녕','형','형님','건','안','요','거','피','질','번','용','군','영','스','좀','데','듯','넉',\
                  '테','런','하다','하나요','되다','안되다','있다','없다','다','더','게','중','전','시','못','뭐','걸','개','격','재','씽','해','큐','내','만','해도','건가',\
                  '점','키','수가','두','도','분','함','통','날','온','적','임','앞','하라','꼭','하니','위','줄','습','볼','뒤','끌','외','곳','G','크롤',\
                  '초','난','세','fi','땐','싹','g','은','wi','원','기','쪽','놈','꽉','되어다','해주다','너무','매우','돼다'])

def rem_n_stopwords(sent_list):
    sent_stop=[]
    for sentence in sent_list:
        new_sent=[]
        for token in sentence:
            if token not in stopwords:
                new_sent.append(token)
        sent_stop.append(new_sent)
    return(sent_stop)

tmp2['text_tokenized2']=tmp2['text_tokenized'].apply(lambda x: rem_n_stopwords(x))

C:\Users\eunah.lee\AppData\Local\Temp/ipykernel_11376/1449202739.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2['text_tokenized2']=tmp2['text_tokenized'].apply(lambda x: rem_n_stopwords(x))


In [64]:
# 빈 리스트 제거
def remove_empty_list(sent_list):
    removed_list = []
    for sentence in sent_list:
        if sentence: # sentence가 빈 경우가 아니면
            removed_list.append(sentence)    
    return removed_list

tmp['text_tokenized2']=tmp['text_tokenized2'].apply(lambda x: remove_empty_list(x))

# review_df.to_csv('thinq_review_result.csv',index=False, encoding='utf-8-sig')

# 리뷰 전체를 한 리스트로 만들기
def one_word_list(sent_list):
    one_list=[]
    for sentence in sent_list:
        one_list.extend(sentence)
    return(one_list)

# test['content_n']=test['content_n'].apply(lambda x: one_word_list(x))
# tmp['text_tokenized'] = tmp['text_tokenized'].apply(lambda x: one_word_list(x))
# tmp['text_tokenized2']=tmp['text_tokenized2'].apply(lambda x: one_word_list(x))

C:\Users\eunah.lee\AppData\Local\Temp/ipykernel_11376/3833982617.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['text_tokenized2']=tmp['text_tokenized2'].apply(lambda x: remove_empty_list(x))


In [71]:
tmp.to_csv('./crawling_data/감성대화말뭉치_ver토큰1_Training.csv', encoding='utf-8 sig')

#### 감성문서 텍스트 추가 전처리  
* 감정_대분류:  
1 = 기쁨  
0 = 불안, 슬픔, 분노, 상처, 불안  
당황은 제거

In [18]:
sa['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [19]:
sa = sa[sa.감정_대분류 != '당황']

In [22]:
sa['bi_sent'] = 0
sa['bi_sent'][(sa['감정_대분류'] == '기쁨')] = 1

C:\Users\eunah.lee\AppData\Local\Temp/ipykernel_23496/3479436705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa['bi_sent'][(sa['감정_대분류'] == '기쁨')] = 1


In [ ]:
sa[(sa['bi_sent']==1)] # 6,730
sa[(sa['bi_sent']==0)] # 27,789

In [30]:
ssa = sa[['감정_대분류','bi_sent']]
X = ssa['감정_대분류']
y = ssa['bi_sent']

In [33]:
train_data, test_data = train_test_split(ssa, test_size = 0.1, shuffle=True,  random_state = 42)

In [35]:
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))
print(train_data.groupby('bi_sent').size().reset_index(name = 'count'))

훈련용 리뷰의 개수 : 31067
테스트용 리뷰의 개수 : 3452
   bi_sent  count
0        0  25016
1        1   6051


In [36]:
train_data

,감정_대분류,bi_sent
36811,슬픔,0
11145,기쁨,1
24277,상처,0
38533,슬픔,0
24586,불안,0
...,...,...
19948,상처,0
7392,슬픔,0
13345,기쁨,1
1026,슬픔,0
